<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024_0524eigenfaces_vs_fisherfaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import PIL.Image as Image
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2, suppress=True)

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

$$
\newcommand{\mb}[1]{\mathbf{#1}}
\newcommand{\Brc}[1]{\left(#1\right)}
\newcommand{\Rank}{\text{rank}\;}
\newcommand{\Hat}[1]{\widehat{#1}}
\newcommand{\Prj}[1]{\mb{#1}\Brc{\mb{#1}^{\top}\mb{#1}}^{-1}\mb{#1}^{\top}}
\newcommand{\RegP}[2]{\Brc{\mb{#1}^{\top}\mb{#1}}^{-1}\mb{#1}^{\top}\mb{#2}}
\newcommand{\NSQ}[1]{\left|\mb{#1}\right|^2}
\newcommand{\Norm}[1]{\left|#1\right|}
\newcommand{\IP}[2]{\left({#1}\cdot{#2}\right)}
\newcommand{\Bar}[1]{\overline{\;#1\;}}
$$


$$\begin{aligned}
\mb{Y}               &= \mb{XW}               & & \text{:オリジナル問題 中学数学 $y=ax$ の一般化}\\
\mb{X}^{\top} \mb{Y} &= \mb{X}^{\top} \mb{XW} & & \text{:両辺に左から $\mb{X}$ の転置行列を掛ける}\\
\left(\mb{X}^{\top} \mb{X}\right)^{-1} \mb{X}^{\top} \mb{Y} &= \left(\mb{X}^{\top} \mb{X}\right)^{-1} \left(\mb{X}^{\top} \mb{X}\right) \mb{W}
& & \text{: 両辺に左から $\mb{X}^{\top}\mb{X}$ の逆行列を掛ける}\\
\left(\mb{X}^{\top} \mb{X}\right)^{-1} \mb{X}^{\top} \mb{Y} &= \mb{W} & & \text{: $\mb{A}^{-1}\mb{A}=\mb{I}$ すなわち単位行列となるので右辺 RHS の $\mb{X}$ に関連する因子は消える}\\
\end{aligned}$$


上式 $\mb{W} = \left(\mb{X}^{\top} \mb{X}\right)^{-1} \mb{X}^{\top} \mb{Y}$ を元の式に代入すると，$\mb{Y}$ の予測行列 $\Hat{\mb{Y}}$ の推定値を得ることができる:

$$\Hat{\mb{Y}} = \mb{X} \left(\mb{X}^{\top} \mb{X}\right)^{-1} \mb{X}^{\top} \mb{Y}
$$

$\mb{X} \left(\mb{X}^{\top} \mb{X}\right)^{-1} \mb{X}^{\top} = \mb{P}$ を $\mb{X}$ で張られる空間への射影行列と呼ぶ。


In [ ]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from sklearn import datasets

data = fetch_olivetti_faces()
D, y = data.data, data.target

X = D - D.mean(axis=0)
Cov = X.T @ X
Cov_inv = np.linalg.inv(Cov)

P = X @ Cov_inv @ X.T
print(f'P.shape:{P.shape}')

Y = np.zeros((len(y),len(set(y))))
for i, _y in enumerate(y):
    Y[i,_y]= 1

print(
    f"X.T.shape:{X.T.shape}\n",
    f"Cov.shape:{Cov.shape}\n",
f"Cov_inv.shape:{Cov_inv.shape}\n",
f"P.shape:{P.shape}\n",
f"Y.shape:{Y.shape}")

In [ ]:
def Prj(D:np.array)->np.array:
    mu = D.mean(axis=0)
    X = D - mu
    Cov = X.T @ X
    Cov_inv = np.linalg.inv(Cov)
    det_Cov = np.linalg.det(Cov)
    P = X @ Cov_inv @ X.T
    return P, Cov, mu, det_Cov

X, y = data.data, data.target
Y = np.zeros((len(y),len(set(y))))
for i, _y in enumerate(y):
    Y[i,_y]= 1

#Eig_val, Eig_vec, mu = pca(X, n_components=5)

In [ ]:
def pca(X, n_components=0):
    [n,d] = X.shape
    if (n_components <= 0) or (n_components>n):
        n_components = n
    mu = X.mean(axis=0)
    X = X - mu

    if n>d:
        Cov = X.T @ X
        [Eig_val, Eig_vec] = np.linalg.eigh(Cov)
    else:
        Cov = X @ X.T
        [Eig_val, Eig_vec] = np.linalg.eigh(Cov)
        Eig_vec = X.T @ Eig_vec
        for i in range(n):
            Eig_vec[:,i] = Eig_vec[:,i]/np.linalg.norm(Eig_vec[:,i])

    # 別解として，特異値分解 SVD を用いて固有値を算出する方法もある。
    # Eig_vec, Eig_val, Var = np.linalg.svd(X.T, full_matrices=False)

    # 固有値の大きい順に並べ替え
    idx = np.argsort(-Eig_val)
    Eig_val = Eig_val[idx]
    Eig_vec = Eig_vec[:,idx]

    # 引数で与えられた成分だけを取り出す
    Eig_val = Eig_val[:n_components].copy()
    Eig_vec = Eig_vec[:,0:n_components].copy()
    return [Eig_val, Eig_vec, mu]

In [ ]:
[E_val, E_vec, mu] = pca(X, n_components=14 ** 2)

In [ ]:
def normalize(X, low, high, dtype=None):
    X = np.asarray(X)
    minX, maxX = np.min(X), np.max(X)

    # normalize to [0...1].
    X = X - float(minX)
    X = X / float((maxX - minX))

    # scale to [low...high].
    X = X * (high-low)
    X = X + low
    if dtype is None:
        return np.asarray(X)
    return np.asarray(X, dtype=dtype)

In [ ]:
E = []
for i in range(64):
#for i in range(min(len(X), 14 * 14)):
    e = E_vec[:,i]
    #e = W[:,i].reshape(X[0].shape)
    E.append(normalize(e,0,255))


In [ ]:
from sklearn.model_selection import train_test_split
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [ ]:
num_features = n_components = 150
num_features = n_components = 49

print(f"上位 {n_components:d} 成分の固有顔を {X_train.shape[0]:d} から抽出")


In [ ]:
nrows = 7   # nrows 人分のデータを表示
ncols = 7
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 7 + j
        fig_axes[i][j].imshow(E_vec[:,x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')

In [ ]:
class AbstractDistance(object):
    def __init__(self, name):
        self._name = name

    def __call__(self,p,q):
        raise NotImplementedError("Every AbstractDistance must implement the __call__ method.")

    @property
    def name(self):
        return self._name

    def __repr__(self):
        return self._name


class EuclideanDistance(AbstractDistance):
    def __init__(self):
        AbstractDistance.__init__(self,"EuclideanDistance")

    def __call__(self, p, q):
        p = np.asarray(p).flatten()
        q = np.asarray(q).flatten()
        return np.sqrt(np.sum(np.power((p-q),2)))


class CosineDistance(AbstractDistance):

        def __init__(self):
            AbstractDistance.__init__(self,"CosineDistance")

        def __call__(self, p, q):
            p = np.asarray(p).flatten()
            q = np.asarray(q).flatten()
            return -np.dot(p.T,q) / (np.sqrt(np.dot(p,p.T)*np.dot(q,q.T)))


In [ ]:
class BaseModel(object):
    def __init__(self,
                 X=None, y=None,
                 dist_metric=EuclideanDistance(),
                 num_components=0):
        self.dist_metric = dist_metric
        self.num_components = 0
        self.Proj = []
        self.W = []
        self.mu = []
        if (X is not None) and (y is not None):
            self.compute(X,y)

        def compute(self, X, y):
            raise NotImplementedError("Every BaseModel must implement the compute method.")

        def predict(self, X):
            minDist = np.finfo('float').max
            minClass = -1
            Q = project(self.W, X.reshape(1,-1), self.mu)
            for i in range(len(self.Proj)):
                dist = self.dist_metric(self.projections[i], Q)
                if dist < minDist:
                    minDist = dist
                    minClass = self.y[i]
            return minClass


class EigenfacesModel(BaseModel):

        def __init__(self,
                     X=None, y=None,
                     dist_metric=EuclideanDistance(),
                     num_components=0):
            super().__init__(
                X=X,
                y=y,
                dist_metric=dist_metric,
                num_components=num_components)

        def compute(self, X, y):
            [D, self.W, self.mu] = pca(asRowMatrix(X),y, self.num_components)

            # store labels
            self.y = y

            # store projections
            for xi in X:
                self.projections.append(project(self.W, xi.reshape(1,-1), self.mu))


class FisherfacesModel(BaseModel):

        def __init__(self,
                     X=None, y=None,
                     dist_metric=EuclideanDistance(),
                     num_components=0):
            super().__init__(
                X=X,
                y=y,
                dist_metric=dist_metric,
                num_components=num_components)

        def compute(self, X, y):
            [D, self.W, self.mu] = fisherfaces(asRowMatrix(X),y, self.num_components)

            # store labels
            self.y = y

            # store projections
            for xi in X:
                self.Proj.append(project(self.W, xi.reshape(1,-1), self.mu))

In [ ]:
def project(W, X, mu=None):
    if mu is None:
        return np.dot(X,W)
    return np.dot(X - mu, W)

def reconstruct(W, Y, mu=None):
    if mu is None:
        return np.dot(Y,W.T)
    return np.dot(Y, W.T) + mu

In [ ]:
def lda(X, y, n_components=0):
    y = np.asarray(y)
    [n,d] = X.shape

    c = np.unique(y)
    if (n_components <= 0) or (n_component>(len(c)-1)):
        n_components = (len(c)-1)
    meanTotal = X.mean(axis=0)

    Sw = np.zeros((d, d), dtype=np.float32)
    Sb = np.zeros((d, d), dtype=np.float32)

    for i in c:
        Xi = X[np.where(y==i)[0],:]
        meanClass = Xi.mean(axis=0)
        Sw = Sw + np.dot((Xi-meanClass).T, (Xi-meanClass))
        Sb = Sb + n * np.dot((meanClass - meanTotal).T, (meanClass - meanTotal))

    Eig_val, Eig_vec = np.linalg.eig(np.linalg.inv(Sw)*Sb)
    idx = np.argsort(-Eig_val.real)
    Eig_val, Eig_vec = Eig_val[idx], Eig_vec[:,idx]
    Eig_val = np.array(Eig_val[0:n_components].real, dtype=np.float32, copy=True)
    Eig_vec = np.array(Eig_vec[0:,0:n_components].real, dtype=np.float32, copy=True)
    return [Eig_val, Eig_vec]


def fisherfaces(X,y,n_components=0):
    y = np.asarray(y)
    [n,d] = X.shape
    c = len(np.unique(y))
    [Eig_val_pca, Eig_vec_pca, mu_pca] = pca(X, (n-c))
    [Eig_val_lda, Eig_vec_lda] = lda(project(Eig_vec_pca, X, mu_pca), y, n_components)
    Eig_vec = np.dot(Eig_vec_pca, Eig_vec_lda)
    return [Eig_val_lda, Eig_vec, mu_pca]

In [ ]:
[D, W, mu] = fisherfaces (X, y)

E = []
for i in range(36):
    e = W[:,i] #.reshape (X[0].shape)
    E.append(normalize(e,0,255))


# plot them and store the plot to
nrows = 6   # nrows 人分のデータを表示
ncols = 6
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 4 + j
        fig_axes[i][j].imshow(E[x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')